In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우: 맑은 고딕
plt.rcParams['axes.unicode_minus'] = False      # 마이너스 기호 깨짐 방지
sns.set(font='Malgun Gothic', style='whitegrid')  # seaborn에도 적용

In [3]:
df = pd.read_csv('F:/데이콘/국민대학교_AI빅데이터_분석_경진대회/data/train.csv', encoding='utf-8')
df.head()

,item_id,year,month,seq,type,hs4,weight,quantity,value
0,DEWLVASR,2022,1,1.0,1,3038,14858.0,0.0,32688.0
1,ELQGMQWE,2022,1,1.0,1,2002,62195.0,0.0,110617.0
2,AHMDUILJ,2022,1,1.0,1,2102,18426.0,0.0,72766.0
3,XIPPENFQ,2022,1,1.0,1,2501,20426.0,0.0,11172.0
4,FTSVTTSR,2022,1,1.0,1,2529,248000.0,0.0,143004.0


In [4]:
sub_sample = pd.read_csv('F:/데이콘/국민대학교_AI빅데이터_분석_경진대회/data/sample_submission.csv', encoding='utf-8')
sub_sample.head()

,leading_item_id,following_item_id,value
0,DEWLVASR,ELQGMQWE,999999999
1,DEWLVASR,AHMDUILJ,999999999
2,DEWLVASR,XIPPENFQ,999999999
3,DEWLVASR,FTSVTTSR,999999999
4,DEWLVASR,XMKRPGLB,999999999


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10836 entries, 0 to 10835
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item_id   10836 non-null  object 
 1   year      10836 non-null  int64  
 2   month     10836 non-null  int64  
 3   seq       10836 non-null  float64
 4   type      10836 non-null  int64  
 5   hs4       10836 non-null  int64  
 6   weight    10836 non-null  float64
 7   quantity  10836 non-null  float64
 8   value     10836 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 762.0+ KB


### pre-processing
- type은 모두 1로 제거
- hs4는 코드로 문자열 처리 후 대분류, 중분류로 분할
- 연월별 seq 1,2,3 모두 합치기
- weight, quantity와 value의 비교를 통해 오류가 있는 데이터 -> 301개 제거...?
- quantity가 0이 많음 -> 열 제거...?

In [6]:
df = df.drop('type', axis=1)
df['hs4'] = df['hs4'].astype('str')
df['large_cat'] = df['hs4'].str[:2]
df['medium_cat'] = df['hs4'].str[2:]
df.head()

,item_id,year,month,seq,hs4,weight,quantity,value,large_cat,medium_cat
0,DEWLVASR,2022,1,1.0,3038,14858.0,0.0,32688.0,30,38
1,ELQGMQWE,2022,1,1.0,2002,62195.0,0.0,110617.0,20,02
2,AHMDUILJ,2022,1,1.0,2102,18426.0,0.0,72766.0,21,02
3,XIPPENFQ,2022,1,1.0,2501,20426.0,0.0,11172.0,25,01
4,FTSVTTSR,2022,1,1.0,2529,248000.0,0.0,143004.0,25,29


In [7]:
# seq이 1이상인 경우 확인해보기
df[df['seq'] > 1].sort_values(['item_id','year','month','seq'])

,item_id,year,month,seq,hs4,weight,quantity,value,large_cat,medium_cat
122,AANGBULD,2022,1,2.0,4810,17625.0,0.0,14276.0,48,10
204,AANGBULD,2022,1,3.0,4810,0.0,0.0,0.0,48,10
456,AANGBULD,2022,2,3.0,4810,67983.0,0.0,52347.0,48,10
617,AANGBULD,2022,3,2.0,4810,69544.0,0.0,53549.0,48,10
705,AANGBULD,2022,3,3.0,4810,0.0,0.0,0.0,48,10
...,...,...,...,...,...,...,...,...,...,...
10290,ZXERAXWP,2025,5,3.0,4802,3814.0,0.0,13889.0,48,02
10451,ZXERAXWP,2025,6,2.0,4802,4548.0,0.0,12981.0,48,02
10537,ZXERAXWP,2025,6,3.0,4802,8450.0,0.0,8634.0,48,02
10709,ZXERAXWP,2025,7,2.0,4802,3999.0,0.0,11566.0,48,02


In [8]:
# seq 합치기
cols_sum = ['weight', 'quantity', 'value']
cols_first = ['hs4', 'large_cat', 'medium_cat']

# 월 단위 집계
df_month = (
    df.groupby(['item_id', 'year', 'month'], as_index=False)
      .agg({**{c: 'first' for c in cols_first},
          **{c: 'sum' for c in cols_sum}},)
)

# seq 개수 다시 계산해서 붙이기 (선택)
df_month['seq'] = df.groupby(['item_id','year','month'])['seq'].count().values

In [9]:
df_month # 3776나오는게 맞는지

,item_id,year,month,hs4,large_cat,medium_cat,weight,quantity,value,seq
0,AANGBULD,2022,1,4810,48,10,17625.0,0.0,14276.0,2
1,AANGBULD,2022,2,4810,48,10,67983.0,0.0,52347.0,1
2,AANGBULD,2022,3,4810,48,10,69544.0,0.0,53549.0,2
3,AANGBULD,2022,5,4810,48,10,34173.0,0.0,26997.0,2
4,AANGBULD,2022,6,4810,48,10,103666.0,0.0,84489.0,3
...,...,...,...,...,...,...,...,...,...,...
3771,ZXERAXWP,2025,3,4802,48,02,3507.0,0.0,12817.0,3
3772,ZXERAXWP,2025,4,4802,48,02,4258.0,0.0,12482.0,3
3773,ZXERAXWP,2025,5,4802,48,02,4598.0,0.0,18224.0,3
3774,ZXERAXWP,2025,6,4802,48,02,36632.0,0.0,42690.0,3


In [10]:
df = df_month.copy()

In [11]:
df.to_csv('train_month.csv', index=False, encoding='utf-8')